In [1]:
from circuit import gen_random_circuits
from upstream import RandomwalkModel
from collections import defaultdict

import math
from utils.backend import gen_grid_topology, get_grid_neighbor_info, Backend
# from utils.backend import default_basis_single_gates, default_basis_two_gates


import pennylane as qml
import numpy as np
from downstream.synthesis.pennylane_op import layer_circuit_to_pennylane_circuit, layer_circuit_to_pennylane_tape
from downstream.synthesis.tensor_network_op import layer_circuit_to_matrix
from downstream.synthesis import matrix_distance_squared
from sklearn.utils import shuffle

import cloudpickle as pickle
from utils.backend import gen_grid_topology, get_grid_neighbor_info, Backend
from circuit import gen_random_circuits
from upstream import RandomwalkModel
# from downstream.synthesis.pennylane_op import layer_circuit_to_pennylane_circuit, layer_circuit_to_pennylane_tape
from downstream.synthesis.tensor_network_op import layer_circuit_to_matrix
# from downstream.synthesis.neural_network import NeuralNetworkModel
from sklearn.neural_network import MLPRegressor as DNN

from jax import numpy as jnp
from jax import vmap
import jax
import optax
from jax.config import config
import pennylane as qml
from pennylane import numpy as pnp
import numpy as np
from scipy.stats import unitary_group
import random
import copy
import ray
from sklearn.model_selection import train_test_split

# ray.init()


grid_num = 2
topology = gen_grid_topology(grid_num) # 3x3 9 qubits
neigh_info = get_grid_neighbor_info(grid_num, 1)


n_qubits = grid_num ** 2
backend = Backend(n_qubits=n_qubits, topology=topology, neighbor_info=neigh_info, basis_single_gates = ['u'], 
                  basis_two_gates = ['cz'], divide = False, decoupling=False)



from downstream.synthesis import SynthesisModel
from scipy.stats import unitary_group
# import warnings
# warnings.filterwarnings("ignore")

2023-02-20 14:23:35.417250: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-20 14:23:35.417384: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-20 14:23:35.417394: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [14]:
def transformU(U):
    '''转成神经网络里面的格式'''
    U = U.reshape(U.size)
    U = jnp.concatenate([U.imag, U.real])
    U = jnp.array(U, dtype = jnp.float64)
    return U

In [4]:
max_gate = 1000 #4**n_qubits
'''生成用于测试的模板电路'''
circuits = gen_random_circuits(min_gate = 100, max_gate = max_gate, gate_num_step = max_gate//50, n_circuits = 20, two_qubit_gate_probs=[4, 8], backend = backend, reverse = False, optimize = True, multi_process = True)

upstream_model = RandomwalkModel(1, 20, backend = backend, travel_directions=('parallel', 'next'))
upstream_model.train(circuits, multi_process = True)

def random_params(layer2gates):
    layer2gates = copy.deepcopy(layer2gates)
    for layer in layer2gates:
        for gate in layer:
            gate['params'] = [
                random.random() * 2 * jnp.pi
                for param in gate['params']
            ]
    return layer2gates

@ray.remote
def _gen_data(circuit_info, index, n_qubits): 
    gate_vecs, Us = [], []
    layer2gates = circuit_info['layer2gates']
    # print(layer2gates)
    for layer_index, layer_gates in enumerate(layer2gates):
        layer_gates = [
            gate
            for gate in layer_gates
            if len(gate['qubits']) == 1
        ]
        if len(layer_gates) == 0:
            continue
        for _ in range(10):
            target_gate_index = random.randint(0, len(layer_gates)-1)
            target_gate_index = layer_gates[target_gate_index]['id']
            target_gate = circuit_info['gates'][target_gate_index]

            gate_vec = circuit_info['vecs'][target_gate_index]
        
            _layer2gates = random_params(layer2gates[layer_index:])
            # TODO: 应该有些可以重复利用才对
            U = layer_circuit_to_matrix(_layer2gates, n_qubits)  #TODO: vmap
            U = transformU(U)
            
            # 似乎根本没有出现过
            # _dists = vmap(matrix_distance_squared, in_axes=(None, 0))(U, jnp.array(U, dtype=jnp.complex128))
            # if jnp.any(_dists < 0.1):
            #     continuez
            
            Us.append(U)
            gate_vecs.append(np.concatenate([jnp.array(target_gate['qubits']), gate_vec], axis=0)  )
            
    return gate_vecs, Us

print('start gen Us -> gate_vectos')
futures = []
for index, circuit_info in enumerate(circuits):
    # upstream_model.vectorize(circuit_info)
    future = _gen_data.remote(circuit_info, index, n_qubits)
    # future = _gen_data(circuit_info, n_qubits)
    futures.append(future)
    
gate_vecs, Us = [], []
for index, future in enumerate(futures):
    if index % 1000 == 0:
        print('finished rate =', index, '/', len(futures))
    if not isinstance(future, tuple):
        future = ray.get(future)
    gate_vecs += future[0]
    Us += future[1]

gate_vecs = jnp.array(gate_vecs, dtype=jnp.float64)
Us = jnp.array(Us, dtype=jnp.float64)
# Us, Us_test, gate_vecs, gate_vecs_test = train_test_split(Us, gate_vecs, test_size=.1)

len(Us), len(gate_vecs), #len(Us_test), len(gate_vecs_test)

finish random circuit generation with 3600 circuits
(remote_train pid=2705441) train:0/360, 0th offest
(remote_train pid=2708976) train:0/360, 720th offest
(remote_train pid=2705441) train:100/360, 0th offest
(remote_train pid=2705435) train:0/360, 360th offest
(remote_train pid=2705441) train:200/360, 0th offest
(remote_train pid=2705418) train:0/360, 1080th offest
(remote_train pid=2708976) train:100/360, 720th offest
(remote_train pid=2705440) train:0/360, 1440th offest
(remote_train pid=2705441) train:300/360, 0th offest
(remote_train pid=2705435) train:100/360, 360th offest
(remote_train pid=2705437) train:0/360, 2160th offest
(remote_train pid=2705423) train:0/360, 1800th offest
(remote_train pid=2705435) train:200/360, 360th offest
(remote_train pid=2705418) train:100/360, 1080th offest
(remote_train pid=2705460) train:0/360, 2880th offest
(remote_train pid=2708976) train:200/360, 720th offest
(remote_train pid=2705435) train:300/360, 360th offest
(remote_train pid=2705444) trai

(_gen_data pid=2705460) 2023-02-20 11:08:10.914277: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
(_gen_data pid=2705460) WARNING:jax._src.lib.xla_bridge:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_gen_data pid=2705440) 2023-02-20 11:08:11.104525: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
(_gen_data pid=2705440) WARNING:jax._src.lib.xla_bridge:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_gen_data pid=2709041) 2023-02-20 11:08:11.094414: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
(_gen_data pid=2709041) WARNING:j

finished rate = 0 / 3600


(_gen_data pid=2709844) 2023-02-20 11:08:19.827462: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
(_gen_data pid=2709844) WARNING:jax._src.lib.xla_bridge:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_gen_data pid=2708176) 2023-02-20 11:08:19.844974: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
(_gen_data pid=2708176) WARNING:jax._src.lib.xla_bridge:No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_gen_data pid=2709488) 2023-02-20 11:08:19.864797: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
(_gen_data pid=2709488) WARNING:j

finished rate = 1000 / 3600
finished rate = 2000 / 3600
finished rate = 3000 / 3600


(2543000, 2543000)

In [6]:
model_path = 'synthesis_model_1.pkl'
# with open(f'./temp_data/{model_path}', 'wb') as file:
#     pickle.dump((upstream_model, Us, gate_vecs), file)
    
with open(f'./temp_data/{model_path}', 'rb') as file:
    upstream_model, Us, gate_vecs = pickle.load(file)
    
upstream_model: RandomwalkModel = upstream_model
Us: jnp.array = Us
gate_vecs: jnp.array = gate_vecs

In [3]:
len(Us), len(gate_vecs)

(2543000, 2543000)

In [4]:
neural_model = DNN((4**n_qubits, upstream_model.max_table_size**2, 2**n_qubits, 2**n_qubits, upstream_model.max_table_size*2), verbose = True, n_iter_no_change=10) #, activation='sigmoid'
neural_model.fit(Us, gate_vecs)
# neural_model.score(transform(Us_test), gate_vecs_test)
    

Iteration 1, loss = 0.08012055
Iteration 2, loss = 0.07845559
Iteration 3, loss = 0.07856213
Iteration 4, loss = 0.07865916
Iteration 5, loss = 0.07874819
Iteration 6, loss = 0.07882974
Iteration 7, loss = 0.07892536
Iteration 8, loss = 0.07900493
Iteration 9, loss = 0.07911803
Iteration 10, loss = 0.07923878
Iteration 11, loss = 0.07935648
Iteration 12, loss = 0.07949335
Iteration 13, loss = 0.07962894
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


In [5]:
model_path = 'neural_model_1.pkl'
with open(f'./temp_data/{model_path}', 'wb') as file:
    pickle.dump(neural_model, file)
    
with open(f'./temp_data/{model_path}', 'rb') as file:
    neural_model = pickle.load(file)
    
neural_model: DNN = neural_model 

In [ ]:
upstream_model.max_table_size

9

In [7]:
def step_function(x):
    return jnp.where(x>0, 1.0, 0.0)

def sigmoid(x):
    return 1 / (1 + jnp.exp(-x))

def final_mapping(mapping, x):
    # print(mapping)
    return sigmoid(mapping['W'] * x + mapping['B'])

def final_mapping_cost(mapping, x, y):
    predictions = final_mapping(mapping, x)
    return optax.l2_loss(predictions, y).sum()

def final_mapping_cost_batch(mapping, X, Y):
    return vmap(final_mapping_cost, in_axes=(None, 0, 0))(mapping, X, Y).sum()

In [12]:
X = neural_model.predict(Us[:20000])[:,1:upstream_model.max_table_size+1]
Y = gate_vecs[:,1:upstream_model.max_table_size+1][:20000]
print(X.shape, Y.shape)

mapping  = {
    'W': jax.random.normal(shape=(upstream_model.max_table_size,), key=jax.random.PRNGKey((random.randint(0, 10))), dtype=jnp.float64),
    'B': jnp.zeros(shape=(upstream_model.max_table_size,), dtype=jnp.float64)
}
opt = optax.adam(learning_rate=1e-2)
opt_state = opt.init(mapping)

X, X_test, Y, Y_test = train_test_split(X, Y, test_size=0.2)

batch_size = 200

best_loss = 1e10
best_mapping = mapping
for epoch in range(10):
    total_loss = 0
    X, Y = shuffle(X, Y)
    for start in range(0, len(X), batch_size):
        # print(start, mapping)
        _X, _Y = X[start: start+batch_size], Y[start: start+batch_size]
        
        loss_value, gradient = jax.value_and_grad(final_mapping_cost_batch)(mapping, _X, _Y)
        updates, opt_state = opt.update(gradient, opt_state, mapping)
        mapping = optax.apply_updates(mapping, updates) 
        total_loss += loss_value    
        
    if epoch%10 ==0:
        test_loss = final_mapping_cost(mapping, X_test, Y_test)
        print('epoch  %i\t| train loss = %.5f\t|  test loss = %.5f' % (epoch, total_loss, test_loss))
    
        if test_loss < best_loss:
            best_loss = test_loss
            best_mapping = mapping
            
            
model_path = 'mapping_1.pkl'
with open(f'./temp_data/{model_path}', 'wb') as file:
    pickle.dump(mapping, file)
    
with open(f'./temp_data/{model_path}', 'rb') as file:
    mapping = pickle.load(file)       


(20000, 11) (20000, 11)
epoch  0	| train loss = 16374.79868	|  test loss = 3383.22694


In [ ]:
# neural_model = NeuralNetworkModel([4**n_qubits, 2**n_qubits, 2**n_qubits, n_qubits, upstream_model.max_table_size + 1])
# neural_model.fit(Us, gate_vecs)

In [26]:
import importlib
from downstream import synthesis
importlib.reload(synthesis)
target_U = unitary_group.rvs(2**n_qubits)

now_U = jnp.array(target_U)
I = jnp.eye(2**n_qubits)
# if nowU is not identity
now_dist = 1

layer2gates_lists = []
while now_dist > 1e-2:
    vec = neural_model.predict([transformU(now_U)])[0]
    device = round(float(vec[0]))
    gate_vec = final_mapping(mapping, vec[1:])
    gate_vec = jnp.where(gate_vec>(1-1e-1), 1, 0)
    # TODO: 
    layer2gates = upstream_model.reconstruct(device, gate_vec)
    # print(layer2gates)
    params =  synthesis.SynthesisModel.find_parmas(n_qubits, layer2gates, now_U)
    layer_U: jnp.array = layer_circuit_to_matrix(layer2gates, n_qubits, params)
    
    now_dist = now_U @ layer_U.T.conj()
    
    layer2gates_lists.append((params, layer2gates))
    now_dist = matrix_distance_squared(now_U, target_U)
    
    print(now_dist)
    

TypeError: 'int' object is not subscriptable

In [18]:
vec

array([[1.84419578, 0.46818063, 0.39372317, 0.43644778, 0.95936261,
        0.14798604, 0.2792584 , 0.04758023, 0.08237355, 0.10970505,
        0.01624282, 0.02887252]])